In [ ]:
'''
Purpose:
To evaluate the final model on the test cohort
'''

In [ ]:
## Import all required libraries
import numpy as np
import os 
import pandas as pd
import tensorflow as tf
import ast
import matplotlib.pyplot as plt
import scikitplot as skplt
from numpy import savetxt, loadtxt
from tensorflow import keras
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
from sklearn.metrics import roc_curve, auc
from sklearn.utils import resample

In [ ]:
## Set current working directory (source for image tiles)
os.chdir("/media/data/Projects/ICCAVSMETS")

In [ ]:
## Set project directory (model-relevant and processed data)
SSDDir = '/home/thomas/Projects/ICCAvsMETS'
model_dir = SSDDir + '/saved_models/Training'
test_dir = SSDDir + '/saved_models/CrossValidation'
FiguresDir = SSDDir+'/Figures/InternalTest/'

In [ ]:
## Define folder of source image tiles: folders should have the following hierachy: */Category/Material/**.jpg
NormalizedTiles = 'Tiles/Normalized'
Sets = 'Tiles/Sets'
TrainingSetDir = 'Tiles/Sets/Train'
TestSetDir = 'Tiles/Sets/Test'

In [ ]:
## Define positive and negative category
PosCategory = 'KolonMet'
NegCategory = 'ICCA'

In [ ]:
## Define lists
PatientNo = []
Category = []
Tilename = []

In [ ]:
## Ungroup the tables for each fold
def ungroup_data_table(DataTable):

    Tilenames_new = []
    Tilenames_flatten = []
    Category_new = []
    PatientNo_new = []
    n = 0

    for i in DataTable['Tilenames']:
        Tilenames_new.append(i)
        for a in range(i.count(', ')+1):
            PatientNo_new.append(DataTable.loc[n, 'PatientNo'])
            Category_new.append(DataTable.loc[n, 'Category'])
        n = n + 1

    Tilenames_flatten = [inner for item in Tilenames_new for inner in ast.literal_eval(item)] 
    Ungrouped_DataTable = pd.DataFrame({'PatientNo': PatientNo_new, 'Category': Category_new, 'Tilenames': Tilenames_flatten, })
    return Ungrouped_DataTable

In [ ]:
## Read Test Set Table
TestTable = pd.read_csv(SSDDir+'/Tables/TestTable.csv')
TestTableTileLevel = pd.read_csv(SSDDir+'/Tables/TestTableTileLevel.csv')

In [ ]:
## Fixed Parameters
num_classes = TestTable['Category'].nunique()
if num_classes == 2:
    num_classes = num_classes-1
num_patients = len(TestTable.index)

In [ ]:
## Variable Parameters
img_height = 300
img_width = 300
IMAGE_SIZE = [img_height, img_width]

In [ ]:
## Define Target Variable
y = TestTable['Category']

In [ ]:
## Create instance of ImageDataGenerator
idg_test = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
## Use flow from dataframe as iterator
test_data_generator = idg_test.flow_from_dataframe(TestTableTileLevel, directory = TestSetDir,
                                                   x_col = "Tilenames", y_col = "Category",
                                                   batch_size = 64,
                                                   target_size = (img_height, img_width),
                                                   class_mode = 'binary', shuffle = False)

In [ ]:
## Load final model
MyModel = keras.models.load_model(test_dir+'/tuned_model_4.h5')

In [ ]:
# Load final model and evaluate accuracy and loss on test set
results = MyModel.evaluate(test_data_generator)
results = dict(zip(MyModel.metrics_names,results))
        
print('Die Accuracy beträgt ' +str(results['accuracy'])+'.')
print('Der Loss beträgt '+str(results['loss'])+'.')
        
tf.keras.backend.clear_session()

In [ ]:
## Write tile-level probalities of test set
predictions = MyModel.predict(test_data_generator)

In [ ]:
## Read Tile and Patient-Cutoff
Thresholds=loadtxt('/home/thomas/Projects/ICCAvsMETS/Tables/Thresholds_CV.csv', dtype=float, delimiter=',').astype(float)

In [ ]:
## Find predicted class and append in list on tile level, use empiric threshold of 0.5
predicted_class = []
for i in predictions:
    if i > 0.5:
        predicted_class.append(PosCategory)
    else:
        predicted_class.append(NegCategory)

In [ ]:
## Create Tile and Patient Level Prediction Tables
PredTableTileLevel = TestTableTileLevel.copy()
PredTableTileLevel['Predictions'] = predictions
PredTableTileLevel['PredictedClass'] = predicted_class

PredTablePatientLevel = PredTableTileLevel.groupby(['PatientNo', 'Category'])['Predictions'].agg(list).reset_index()
PredTablePatientLevel['Predictions_mean'] = PredTablePatientLevel['Predictions'].apply(np.mean)

In [ ]:
## Find predicted class and append in list on patient level, use cross-validation adjusted threshold
predictionsmean = PredTablePatientLevel['Predictions_mean']
predicted_class2 = []
for i in predictionsmean:
    if i > Thresholds[1]:
        predicted_class2.append(PosCategory)
    else:
        predicted_class2.append(NegCategory)
PredTablePatientLevel['PredictedClass'] = predicted_class2

In [ ]:
##Bootstrapping on patient level
n_iterations = 100
stats = list()
stats2 = list()
accuracy_list1 = []
sensitivity_list1 = []
specificity_list1 = []
ppv_list1 = []
npv_list1 = []
accuracy_list2 = []
sensitivity_list2 = []
specificity_list2 = []
ppv_list2 = []
npv_list2 = []

mean_fpr = np.linspace(0, 1, 100)
tprs = []
tprs2 = []

for i in range(n_iterations):
    boot_subset_patient = resample(TestTable)
    boot_subset_patient.reset_index(drop=True, inplace=True)
    boot_subset = ungroup_data_table(boot_subset_patient)
    boot_data_generator = idg_test.flow_from_dataframe(boot_subset, directory = TestSetDir,
                                                   x_col = "Tilenames", y_col = "Category",
                                                   batch_size = 64,
                                                   target_size = (img_height, img_width),
                                                   class_mode = 'binary', shuffle = False)
    predictions = MyModel.predict(boot_data_generator)
    fpr2, tpr2, thresholds2 = roc_curve(boot_subset['Category'], predictions, pos_label=PosCategory)
    roc_auc = auc(fpr2, tpr2)
    stats.append(roc_auc)
    interp_tpr = np.interp(mean_fpr, fpr2, tpr2)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
        
    predicted_class = []
    for i in predictions:
        if i > 0.5:
            predicted_class.append(PosCategory)
        else:
            predicted_class.append(NegCategory)
    
    boot_subset['Predictions'] = predictions
    boot_subset['PredictedClass'] = predicted_class
    
    KolonMet_TileNo = boot_subset.loc[boot_subset['Category'] == 'KolonMet'].shape[0]
    KolonMet_correct = boot_subset.loc[(boot_subset['Category'] == 'KolonMet') & (boot_subset['PredictedClass'] == 'KolonMet')].shape[0]
    KolonMet_allPositive = boot_subset.loc[boot_subset['PredictedClass'] == 'KolonMet'].shape[0]
    KolonMet_allNegative = boot_subset.loc[boot_subset['PredictedClass'] == 'ICCA'].shape[0]
    KolonMet_correctneg = boot_subset.loc[(boot_subset['Category'] == 'ICCA') & (boot_subset['PredictedClass'] == 'ICCA')].shape[0]

    ICCA_TileNo = boot_subset.loc[boot_subset['Category'] == 'ICCA'].shape[0]
    ICCA_correct = boot_subset.loc[(boot_subset['Category'] == 'ICCA') & (boot_subset['PredictedClass'] == 'ICCA')].shape[0]

    accuracy_list1.append(((KolonMet_correct+ICCA_correct)/(KolonMet_TileNo+ICCA_TileNo))*100)
    sensitivity_list1.append((KolonMet_correct/KolonMet_TileNo)*100)
    specificity_list1.append((ICCA_correct/ICCA_TileNo)*100)
    ppv_list1.append((KolonMet_correct/KolonMet_allPositive)*100)
    npv_list1.append((KolonMet_correctneg/KolonMet_allNegative)*100)
            
    TableCopy = boot_subset.copy()
    TableCopy = TableCopy.groupby(['PatientNo', 'Category'])['Predictions'].agg(list).reset_index()
    TableCopy['Predictions_mean'] = TableCopy['Predictions'].apply(np.mean)
    fpr3, tpr3, thresholds3 = roc_curve(TableCopy['Category'], TableCopy['Predictions_mean'], pos_label=PosCategory)
    roc_auc2 = auc(fpr3, tpr3)
    stats2.append(roc_auc2)
    interp_tpr2 = np.interp(mean_fpr, fpr3, tpr3)
    interp_tpr2[0] = 0.0
    tprs2.append(interp_tpr2)
    
    predicted_class2 = []
    predictionsmean = TableCopy['Predictions_mean']
    for i in predictionsmean:
        if i > Thresholds[1]:
            predicted_class2.append(PosCategory)
        else:
            predicted_class2.append(NegCategory)
    TableCopy['PredictedClass'] = predicted_class2
    
    KolonMet_PatientNo = TableCopy.loc[TableCopy['Category'] == 'KolonMet'].shape[0]
    KolonMet_correct = TableCopy.loc[(TableCopy['Category'] == 'KolonMet') & (TableCopy['PredictedClass'] == 'KolonMet')].shape[0]
    KolonMet_allPositive = TableCopy.loc[TableCopy['PredictedClass'] == 'KolonMet'].shape[0]
    KolonMet_allNegative = TableCopy.loc[TableCopy['PredictedClass'] == 'ICCA'].shape[0]
    KolonMet_correctneg = TableCopy.loc[(TableCopy['Category'] == 'ICCA') & (TableCopy['PredictedClass'] == 'ICCA')].shape[0]

    ICCA_PatientNo = TableCopy.loc[TableCopy['Category'] == 'ICCA'].shape[0]
    ICCA_correct = TableCopy.loc[(TableCopy['Category'] == 'ICCA') & (TableCopy['PredictedClass'] == 'ICCA')].shape[0]

    accuracy_list2.append(((KolonMet_correct+ICCA_correct)/(KolonMet_PatientNo+ICCA_PatientNo))*100)
    sensitivity_list2.append((KolonMet_correct/KolonMet_PatientNo)*100)
    specificity_list2.append((ICCA_correct/ICCA_PatientNo)*100)
    ppv_list2.append((KolonMet_correct/KolonMet_allPositive)*100)
    npv_list2.append((KolonMet_correctneg/KolonMet_allNegative)*100)

## Calculate AUC Confidence Interval
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats, p))

p = ((1.0-alpha)/2.0) * 100
lower2 = max(0.0, np.percentile(stats2, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper2 = min(1.0, np.percentile(stats2, p))

## Calculate ROC curve confidence boundaries
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
roc_lower = np.percentile(tprs, p, axis=0)
p = (alpha+((1.0-alpha)/2.0)) * 100
roc_upper = np.percentile(tprs, p, axis=0)

alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
roc_lower2 = np.percentile(tprs2, p, axis=0)
p = (alpha+((1.0-alpha)/2.0)) * 100
roc_upper2 = np.percentile(tprs2, p, axis=0)

In [ ]:
##ROC auf Tile-Level
fpr2, tpr2, thresholds2 = roc_curve(PredTableTileLevel['Category'], PredTableTileLevel['Predictions'], pos_label=PosCategory)
auc_tile = auc(fpr2, tpr2)
plt.figure(2)
plt.plot([0, 1], [0, 1], 'k--', linewidth = 1.0)
plt.plot(fpr2, tpr2, linewidth = 1.0, label='AUC = ' + str(format((round(auc_tile,3)),'.3f')), zorder=3)
plt.fill_between(mean_fpr, roc_lower, roc_upper, color='moccasin',
                 label='95% CI ' +str(format((round((lower),3)),'.3f')) + '-' + str(format((round((upper),3)),'.3f')), zorder=1)
plt.xlabel('False positive rate', fontsize=12, fontweight='bold')
plt.ylabel('True positive rate', fontsize=12, fontweight='bold')
plt.tick_params(axis='both', which='major', labelsize=10)
plt.tick_params(axis='both', which='minor', labelsize=10)
plt.xlim(0,1)
plt.ylim(0,1)
plt.gca().set_aspect('equal', adjustable='box')
leg = plt.legend(loc='lower right', fontsize=8)
leg.get_frame().set_linewidth(0.0)
plt.gca().spines['left'].set_zorder(2)
plt.gca().spines['top'].set_zorder(2)
plt.savefig(FiguresDir+'ROC_Test_TileLV.png', dpi=1200, bbox_inches='tight')
plt.show()
print('Die AUC auf Tile-Level beträgt '+str(round(auc_tile,2))+'.')

In [ ]:
## Determine optimal threshold on tile level
optimal_idx = np.argmax(tpr2 - fpr2)
optimal_threshold_TileLevel = thresholds2[optimal_idx]

In [ ]:
## Plot ROC-Curve for Patient Level
fpr, tpr, thresholds = roc_curve(PredTablePatientLevel['Category'], PredTablePatientLevel['Predictions_mean'], pos_label=PosCategory)
auc_patient = auc(fpr, tpr)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--', linewidth = 1.0)
plt.plot(fpr, tpr, linewidth = 1.0, label='AUC = ' + str(format((round(auc_patient,3)),'.3f')), zorder=3)
plt.fill_between(mean_fpr, roc_lower2, roc_upper2, color='moccasin',
                 label='95% CI ' + str(format((round((lower2),3)),'.3f')) + '-' + str(format((round((upper2),3)),'.3f')), zorder=1)
plt.xlabel('False positive rate', fontsize=12, fontweight='bold')
plt.ylabel('True positive rate', fontsize=12, fontweight='bold')
plt.tick_params(axis='both', which='major', labelsize=10)
plt.tick_params(axis='both', which='minor', labelsize=10)
plt.xlim(0,1)
plt.ylim(0,1)
plt.gca().set_aspect('equal', adjustable='box')
leg = plt.legend(loc='lower right', fontsize=8)
leg.get_frame().set_linewidth(0.0)
plt.gca().spines['left'].set_zorder(2)
plt.gca().spines['top'].set_zorder(2)
plt.savefig(FiguresDir+'ROC_Test_PatientLV.png', dpi=1200, bbox_inches='tight')
plt.show()
print('Die AUC auf Patient-Level beträgt '+str(round(auc_patient,2))+'.')

In [ ]:
## Determine optimal threshold on patient level
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold_PatientLevel = thresholds[optimal_idx]

In [ ]:
## Save thresholds
Thresholds_Test = np.asarray([optimal_threshold_TileLevel, optimal_threshold_PatientLevel])
savetxt('/home/thomas/Projects/ICCAvsMETS/Tables/Thresholds_Test.csv', Thresholds_Test, delimiter=',')

In [ ]:
# Generate confusion matrix on patient level (absolute)
skplt.metrics.plot_confusion_matrix(PredTablePatientLevel['Category'], PredTablePatientLevel['PredictedClass'], title = ' ', figsize = (4,3),normalize=False)
plt.xlabel('Predicted', fontweight='bold')
plt.ylabel('Ground Truth', fontweight='bold')
locs, labels = plt.xticks() 
plt.xticks(locs,['iCCA', 'CRM'])
locs, labels = plt.yticks() 
plt.yticks(locs,['iCCA', 'CRM'])
plt.savefig(FiguresDir+'CoMa_Test_PatientLV_abs.png', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
# Generate confusion matrix on patient level (relative)
skplt.metrics.plot_confusion_matrix(PredTablePatientLevel['Category'], PredTablePatientLevel['PredictedClass'], title = ' ', figsize = (4,3),normalize=True)
plt.xlabel('Predicted', fontweight='bold')
plt.ylabel('Ground Truth', fontweight='bold')
locs, labels = plt.xticks() 
plt.xticks(locs,['iCCA', 'CRM'])
locs, labels = plt.yticks() 
plt.yticks(locs,['iCCA', 'CRM'])
plt.savefig(FiguresDir+'CoMa_Test_PatientLV_rel.png', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
# Generate confusion matrix on tile level (absolute)
skplt.metrics.plot_confusion_matrix(PredTableTileLevel['Category'], PredTableTileLevel['PredictedClass'], title = ' ', figsize = (4,3), normalize=False)
plt.xlabel('Predicted', fontweight='bold')
plt.ylabel('Ground Truth', fontweight='bold')
locs, labels = plt.xticks() 
plt.xticks(locs,['iCCA', 'CRM'])
locs, labels = plt.yticks() 
plt.yticks(locs,['iCCA', 'CRM'])
plt.savefig(FiguresDir+'CoMa_Test_TileLV_abs.png', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
# Generate confusion matrix on tile level (relative)
skplt.metrics.plot_confusion_matrix(PredTableTileLevel['Category'], PredTableTileLevel['PredictedClass'], title = ' ', figsize = (4,3), normalize=True)
plt.xlabel('Predicted', fontweight='bold')
plt.ylabel('Ground Truth', fontweight='bold')
locs, labels = plt.xticks() 
plt.xticks(locs,['iCCA', 'CRM'])
locs, labels = plt.yticks() 
plt.yticks(locs,['iCCA', 'CRM'])
plt.savefig(FiguresDir+'CoMa_Test_TileLV_rel.png', dpi=1200, bbox_inches='tight')
plt.show()

In [ ]:
## Save prediction tables
PredTablePatientLevel.to_csv('/home/thomas/Projects/ICCAvsMETS/Tables/PredTablePatientLevel.csv', index=False)
PredTableTileLevel.to_csv('/home/thomas/Projects/ICCAvsMETS/Tables/PredTableTileLevel.csv', index=False)

In [ ]:
##Compute metrics on tile level. Arbritarly, colorectal metastasis is defined as disease.  
Metrics_TileLevel_Test = pd.DataFrame(columns=['Name', 'Accuracy','Sensitivity','Specificity','PPV','NPV'])
names = ['Original','Lower bound (95%CI)', 'Upper bound (95%CI)']
Metrics_TileLevel_Test['Name'] = names
accuracy=[]
sensitivity=[]
specificity=[]
ppv =[]
npv=[]

KolonMet_TileNo = PredTableTileLevel.loc[PredTableTileLevel['Category'] == 'KolonMet'].shape[0]
KolonMet_correct = PredTableTileLevel.loc[(PredTableTileLevel['Category'] == 'KolonMet') & (PredTableTileLevel['PredictedClass'] == 'KolonMet')].shape[0]
KolonMet_allPositive = PredTableTileLevel.loc[PredTableTileLevel['PredictedClass'] == 'KolonMet'].shape[0]
KolonMet_allNegative = PredTableTileLevel.loc[PredTableTileLevel['PredictedClass'] == 'ICCA'].shape[0]
KolonMet_correctneg = PredTableTileLevel.loc[(PredTableTileLevel['Category'] == 'ICCA') & (PredTableTileLevel['PredictedClass'] == 'ICCA')].shape[0]

ICCA_TileNo = PredTableTileLevel.loc[PredTableTileLevel['Category'] == 'ICCA'].shape[0]
ICCA_correct = PredTableTileLevel.loc[(PredTableTileLevel['Category'] == 'ICCA') & (PredTableTileLevel['PredictedClass'] == 'ICCA')].shape[0]

accuracy.append(np.round(((KolonMet_correct+ICCA_correct)/(KolonMet_TileNo+ICCA_TileNo))*100,3))
sensitivity.append(np.round((KolonMet_correct/KolonMet_TileNo)*100,3))
specificity.append(np.round((ICCA_correct/ICCA_TileNo)*100,3))
ppv.append(np.round((KolonMet_correct/KolonMet_allPositive)*100,3))
npv.append(np.round((KolonMet_correctneg/KolonMet_allNegative)*100,3))

alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
p1 = (alpha+((1.0-alpha)/2.0)) * 100

accuracy.append(np.round(max(0.0, np.percentile(accuracy_list1, p)),3))
sensitivity.append(np.round(max(0.0, np.percentile(sensitivity_list1, p)),3))
specificity.append(np.round(max(0.0, np.percentile(specificity_list1, p)),3))
ppv.append(np.round(max(0.0, np.percentile(ppv_list1, p)),3))
npv.append(np.round(max(0.0, np.percentile(npv_list1, p)),3))

accuracy.append(np.round(min(100.0, np.percentile(accuracy_list1, p1)),3))
sensitivity.append(np.round(min(100.0, np.percentile(sensitivity_list1, p1)),3))
specificity.append(np.round(min(100.0, np.percentile(specificity_list1, p1)),3))
ppv.append(np.round(min(100.0,np.percentile(ppv_list1, p1)),3))
npv.append(np.round(min(100.0, np.percentile(npv_list1, p1)),3))

Metrics_TileLevel_Test['Accuracy']=accuracy
Metrics_TileLevel_Test['Sensitivity']=sensitivity
Metrics_TileLevel_Test['Specificity']=specificity
Metrics_TileLevel_Test['PPV']=ppv
Metrics_TileLevel_Test['NPV']=npv

In [ ]:
## Save tile level metrics 
Metrics_TileLevel_Test.to_csv('/home/thomas/Projects/ICCAvsMETS/Tables/Metrics_TileLevel_Test.csv', index=False)

In [ ]:
## Save bootstrap lists as pandas dataframes for reproducibility, on tile level
Bootstrap_internal_TileLevel = pd.DataFrame(columns=['Accuracy', 'Sensitivity','Specificity','PPV','NPV', 'AUC'])
Bootstrap_internal_TileLevel['Accuracy']=accuracy_list1
Bootstrap_internal_TileLevel['Sensitivity']=sensitivity_list1
Bootstrap_internal_TileLevel['Specificity']=specificity_list1
Bootstrap_internal_TileLevel['PPV']=ppv_list1
Bootstrap_internal_TileLevel['NPV']=npv_list1
Bootstrap_internal_TileLevel['AUC']=stats

Bootstrap_internal_TileLevel.to_csv('/home/thomas/Projects/ICCAvsMETS/Tables/Bootstrap_internal_TileLevel.csv', index=False)

In [ ]:
##Compute metrics on patient level. Arbritarly, colorectal metastasis is defined as disease. 
Metrics_PatientLevel_Test = pd.DataFrame(columns=['Name', 'Accuracy','Sensitivity','Specificity','PPV','NPV'])
names = ['Original','Lower bound (95%CI)', 'Upper bound (95%CI)']
Metrics_PatientLevel_Test['Name'] = names
accuracy=[]
sensitivity=[]
specificity=[]
ppv =[]
npv=[]

KolonMet_PatientNo = PredTablePatientLevel.loc[PredTablePatientLevel['Category'] == 'KolonMet'].shape[0]
KolonMet_correct = PredTablePatientLevel.loc[(PredTablePatientLevel['Category'] == 'KolonMet') & (PredTablePatientLevel['PredictedClass'] == 'KolonMet')].shape[0]
KolonMet_allPositive = PredTablePatientLevel.loc[PredTablePatientLevel['PredictedClass'] == 'KolonMet'].shape[0]
KolonMet_allNegative = PredTablePatientLevel.loc[PredTablePatientLevel['PredictedClass'] == 'ICCA'].shape[0]
KolonMet_correctneg = PredTablePatientLevel.loc[(PredTablePatientLevel['Category'] == 'ICCA') & (PredTablePatientLevel['PredictedClass'] == 'ICCA')].shape[0]

ICCA_PatientNo = PredTablePatientLevel.loc[PredTablePatientLevel['Category'] == 'ICCA'].shape[0]
ICCA_correct = PredTablePatientLevel.loc[(PredTablePatientLevel['Category'] == 'ICCA') & (PredTablePatientLevel['PredictedClass'] == 'ICCA')].shape[0]

accuracy.append(np.round(((KolonMet_correct+ICCA_correct)/(KolonMet_PatientNo+ICCA_PatientNo))*100,3))
sensitivity.append(np.round((KolonMet_correct/KolonMet_PatientNo)*100,3))
specificity.append(np.round((ICCA_correct/ICCA_PatientNo)*100,3))
ppv.append(np.round((KolonMet_correct/KolonMet_allPositive)*100,3))
npv.append(np.round((KolonMet_correctneg/KolonMet_allNegative)*100,3))

alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
p1 = (alpha+((1.0-alpha)/2.0)) * 100

accuracy.append(np.round(max(0.0, np.percentile(accuracy_list2, p)),3))
sensitivity.append(np.round(max(0.0, np.percentile(sensitivity_list2, p)),3))
specificity.append(np.round(max(0.0, np.percentile(specificity_list2, p)),3))
ppv.append(np.round(max(0.0, np.percentile(ppv_list2, p)),3))
npv.append(np.round(max(0.0, np.percentile(npv_list2, p)),3))

accuracy.append(np.round(min(100.0, np.percentile(accuracy_list2, p1)),3))
sensitivity.append(np.round(min(100.0, np.percentile(sensitivity_list2, p1)),3))
specificity.append(np.round(min(100.0, np.percentile(specificity_list2, p1)),3))
ppv.append(np.round(min(100.0,np.percentile(ppv_list2, p1)),3))
npv.append(np.round(min(100.0, np.percentile(npv_list2, p1)),3))

Metrics_PatientLevel_Test['Accuracy']=accuracy
Metrics_PatientLevel_Test['Sensitivity']=sensitivity
Metrics_PatientLevel_Test['Specificity']=specificity
Metrics_PatientLevel_Test['PPV']=ppv
Metrics_PatientLevel_Test['NPV']=npv

In [ ]:
## Save tile level metrics
Metrics_PatientLevel_Test.to_csv('/home/thomas/Projects/ICCAvsMETS/Tables/Metrics_PatientLevel_Test.csv', index=False)

In [ ]:
## Save bootstrap lists as Pandas dataframes for reproducibility, on patient level
Bootstrap_internal_PatientLevel = pd.DataFrame(columns=['Accuracy', 'Sensitivity','Specificity','PPV','NPV', 'AUC'])
Bootstrap_internal_PatientLevel['Accuracy']=accuracy_list2
Bootstrap_internal_PatientLevel['Sensitivity']=sensitivity_list2
Bootstrap_internal_PatientLevel['Specificity']=specificity_list2
Bootstrap_internal_PatientLevel['PPV']=ppv_list2
Bootstrap_internal_PatientLevel['NPV']=npv_list2
Bootstrap_internal_PatientLevel['AUC']=stats2

Bootstrap_internal_PatientLevel.to_csv('/home/thomas/Projects/ICCAvsMETS/Tables/Bootstrap_internal_PatientLevel.csv', index=False)